In [ ]:
# %run ./Binance_keys.ipynb

In [ ]:
import sqlalchemy
import pandas as pd
from binance import Client

In [ ]:
api_key = ''
api_secret = ''

In [ ]:
client = Client(api_key, api_secret)

In [ ]:
engine = sqlalchemy.create_engine('sqlite:///ETHUSDTstream.db')

In [ ]:
df = pd.read_sql('ETHUSDT', engine)

In [ ]:
df

In [ ]:
df.Price.plot()

In [ ]:
def buy_quantity(asset):
    if asset == 'USDT':
        balance = client.get_asset_balance(asset = asset)
        portion_balance = float(balance['free']) / df.Price.iloc[-1] * 0.80
    elif asset == 'ETH':
        balance = client.get_asset_balance(asset = asset)
        portion_balance = float(balance['free']) * 0.80
    return round(portion_balance, 3)

In [ ]:
def strategy(entry, lookback, open_position=False):
    while True:
        df = pd.read_sql('ETHUSDT', engine)
        lookbackperiod = df.iloc[-lookback:]
        cumret = (lookbackperiod.Price.pct_change() + 1).cumprod() - 1
        if not open_position:
            if cumret[cumret.last_valid_index()] > entry:
                order = client.create_order(symbol='ETHUSDT', side='BUY', type='MARKET', quantity=buy_quantity('USDT'))
                print(order)
                open_position = True
                break
    if open_position:
        while True:
            df = pd.read_sql('ETHUSDT', engine)
            sincebuy = df.loc[df.Time > pd.to_datetime(order['transactTime'], unit='ms')]
            if len(sincebuy) > 1:
                sincebuyret = (sincebuy.Price.pct_change() + 1).cumprod() - 1
                last_entry = sincebuyret[sincebuyret.last_valid_index()]
                if last_entry > 0.0019:
                    order = client.create_order(symbol='ETHUSDT', side='SELL', type='MARKET', quantity=buy_quantity('ETH'))
                    print(order)
                    break

In [ ]:
strategy(0.001, 60) 

In [ ]:
info = client.get_symbol_info('ETHUSDT')
print(info)